<a href="https://colab.research.google.com/github/PavleSavic/MLM_consistency/blob/main/consistency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import string
import logging
from typing import Callable
import pandas as pd
import numpy as np
import tensorflow as tf
#!pip install transformers datasets evaluate
from transformers import AutoTokenizer, TFAutoModelForMaskedLM, TFAutoModel

In [ ]:
random.seed(123)
tf.keras.mixed_precision.set_global_policy('mixed_float16')
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


## Relations used in analysis

In [ ]:
relations = []
with open("final_19_relations.txt") as f:
    lines = f.readlines()
    for l in lines:
        relations.append(l.strip())
print(len(relations))
relations.sort()
print(relations)

19
['associated_morphology_of', 'disease_has_abnormal_cell', 'disease_has_associated_anatomic_site', 'disease_has_normal_cell_origin', 'disease_has_normal_tissue_origin', 'disease_mapped_to_gene', 'disease_may_have_associated_disease', 'disease_may_have_finding', 'disease_may_have_molecular_abnormality', 'gene_associated_with_disease', 'gene_encodes_gene_product', 'gene_product_encoded_by_gene', 'gene_product_has_associated_anatomy', 'gene_product_has_biochemical_function', 'gene_product_plays_role_in_biological_process', 'has_physiologic_effect', 'may_prevent', 'may_treat', 'occurs_after']


## Prompts

In [ ]:
prompts = pd.read_csv('prompts.csv')

In [ ]:
prompts

,pid,default_prompt,human_prompt
0,associated_morphology_of,[X] associated morphology of [Y] .,[X] is associated morphology of [Y] .
1,disease_has_abnormal_cell,[X] disease has abnormal cell [Y] .,[X] has the abnormal cell [Y] .
2,disease_has_associated_anatomic_site,[X] disease has associated anatomic site [Y] .,The disease [X] can stem from the associated a...
3,disease_has_normal_cell_origin,[X] disease has normal cell origin [Y] .,The disease [X] stems from the normal cell [Y] .
4,disease_has_normal_tissue_origin,[X] disease has normal tissue origin [Y] .,The disease [X] stems from the normal tissue [...
5,disease_mapped_to_gene,[X] disease mapped to gene [Y] .,The disease [X] is mapped to gene [Y] .
6,disease_may_have_associated_disease,[X] disease may have associated disease [Y] .,The disease [X] might have the associated dise...
7,disease_may_have_finding,[X] disease may have finding [Y] .,[X] may have [Y] .
8,disease_may_have_molecular_abnormality,[X] disease may have molecular abnormality [Y] .,The disease [X] may have molecular abnormality...
9,gene_associated_with_disease,[X] gene associated with disease [Y] .,The gene [X] is associatied with disease [Y] .


## Masked Language Models

In [ ]:
# uncased
bert_models = {'BERT_base' : "google-bert/bert-base-uncased", 'BERT_large': "google-bert/bert-large-uncased",
                'BERT_large_wwm': "google-bert/bert-large-uncased-whole-word-masking"}
# cased
roberta_models = {'RoBERTa_base': "FacebookAI/roberta-base", 'RoBERTa_large': "FacebookAI/roberta-large"}
# uncased
albert_models = {'ALBERT_base': "albert/albert-base-v2", 'ALBERT_xxlarge': "albert/albert-xxlarge-v2"}
# cased
biobert_models = {'BioBERT': "dmis-lab/biobert-base-cased-v1.2"}
# uncased
biomedbert_models = {'BioMedBERT_base_abstract' : "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract",
                     'BioMedBERT_base_full': "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext",
                     'BioMedBERT_large_abstract': "microsoft/BiomedNLP-BiomedBERT-large-uncased-abstract"}

## Example

In [ ]:
# List of input texts with masked tokens
texts = ["She is from the city of [MASK].", "This is a great [MASK].", "He is an excellent [MASK]."]

In [ ]:
def change_input_format(input):
  new_input = input.replace('[MASK]','<mask>')
  return new_input

In [ ]:
def analyze_tokenizer(model_checkpoint:str, inputs):

  if 'roberta' in model_checkpoint:
    inputs = [change_input_format(input) for input in inputs]

  tokz = AutoTokenizer.from_pretrained(model_checkpoint)

  # model_max_length field not set by default for BioBERT and BioMedBERT models
  if 'bio' in model_checkpoint.lower():
    tokz.model_max_length = 512

  tokenization = tokz(inputs, return_tensors='tf', padding=True)  # truncation=True, max_length=tokz.model_max_length
  print(f"Tokenization example: {tokenization['input_ids']}")
  for l in tokenization['input_ids']:
    print(f"Decoded tokens: {tokz.decode(l)}")
  print(f"End of sequence token: {tokz.eos_token}")
  print(f"Mask token id: {tokz.mask_token_id}")
  print(f"All special tokens ids: {tokz.all_special_ids}")
  print(f"All special tokens: {tokz.decode(tokz.all_special_ids)}")
  print(f"Maximum model input length: {tokz.model_max_length}")

In [ ]:
# One model for each group
print('BERT_base')
analyze_tokenizer(bert_models['BERT_base'], texts)
print('------------------------------------------------------------------------------')
print('BERT_large_wwm')
analyze_tokenizer(bert_models['BERT_large_wwm'], texts)
print('------------------------------------------------------------------------------')
print('RoBERTa_base')
analyze_tokenizer(roberta_models['RoBERTa_base'], texts)
print('------------------------------------------------------------------------------')
print('ALBERT_base')
analyze_tokenizer(albert_models['ALBERT_base'], texts)
print('------------------------------------------------------------------------------')
print('BioBERT')
analyze_tokenizer(biobert_models['BioBERT'], texts)
print('------------------------------------------------------------------------------')
print('BioMedBERT_base_full')
analyze_tokenizer(biomedbert_models['BioMedBERT_base_full'], texts)
print('------------------------------------------------------------------------------')

BERT_base
Tokenization example: [[ 101 2016 2003 2013 1996 2103 1997  103 1012  102]
 [ 101 2023 2003 1037 2307  103 1012  102    0    0]
 [ 101 2002 2003 2019 6581  103 1012  102    0    0]]
Decoded tokens: [CLS] she is from the city of [MASK]. [SEP]
Decoded tokens: [CLS] this is a great [MASK]. [SEP] [PAD] [PAD]
Decoded tokens: [CLS] he is an excellent [MASK]. [SEP] [PAD] [PAD]
End of sequence token: None
Mask token id: 103
All special tokens ids: [100, 102, 0, 101, 103]
All special tokens: [UNK] [SEP] [PAD] [CLS] [MASK]
Maximum model input length: 512
------------------------------------------------------------------------------
BERT_large_wwm
Tokenization example: [[ 101 2016 2003 2013 1996 2103 1997  103 1012  102]
 [ 101 2023 2003 1037 2307  103 1012  102    0    0]
 [ 101 2002 2003 2019 6581  103 1012  102    0    0]]
Decoded tokens: [CLS] she is from the city of [MASK]. [SEP]
Decoded tokens: [CLS] this is a great [MASK]. [SEP] [PAD] [PAD]
Decoded tokens: [CLS] he is an excellen

In [ ]:
def get_model_predictions(model_checkpoint:str, inputs:list[str], top_n=5, verbose=0):
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
  model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint, from_pt=True)

  # Adjusting inputs for RoBERTa models
  if 'roberta' in model_checkpoint:
    inputs = [change_input_format(input) for input in inputs]

  # model_max_length field not set by default for BioBERT and BioMedBERT models
  if 'bio' in model_checkpoint.lower():
    tokenizer.model_max_length = 512

  if verbose:
    print(f'Choosen model: {model_checkpoint}')
    model.summary()

  # Tokenizing the inputs
  tokenized_inputs = tokenizer(inputs, return_tensors="tf", padding=True, truncation=True) # max_length=128 (by default max_length = tokenizer.model_max_length)

  # Getting the token logits from the model
  token_logits = model(**tokenized_inputs).logits

  outputs = []

  for i, input in enumerate(inputs):
                                                                                   # not necessary (all tokenizers have mask_token_id defined)
    mask_token_id = tokenizer.mask_token_id if tokenizer.mask_token_id is not None else tokenizer.convert_tokens_to_ids(["[MASK]"])[0]

    mask_token_index = np.argwhere(tokenized_inputs["input_ids"].numpy()[i] == mask_token_id)[0, 0]

    mask_token_logits = token_logits[i, mask_token_index, :]

    top_tokens = np.argsort(-mask_token_logits.numpy()).tolist()

    predictions = []
    if verbose:
      print(f"Input: {input}")

    for token_id in top_tokens:
        # Skip special tokens
        if token_id in tokenizer.all_special_ids:
          continue

        predicted_token = tokenizer.decode([token_id])

        # subword tokens (???)

        # Skip punctutation tokens
        if predicted_token in string.punctuation:
          continue

        predictions.append(predicted_token)
        if verbose:
          print(f">>> {input.replace(tokenizer.mask_token, predicted_token)}")

        if len(predictions) == top_n:
          break
    if verbose:
      print()

    outputs.append(predictions)

  return np.array(outputs)

In [ ]:
pred = get_model_predictions(model_checkpoint=roberta_models['RoBERTa_base'], inputs=texts, top_n=10)
pred

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

array([[' Chicago', ' London', ' Toronto', ' Seattle', ' Boston',
        ' Portland', ' Philadelphia', ' Vancouver', ' Minneapolis',
        ' Houston'],
       [' idea', ' example', ' article', ' video', ' post', ' story',
        ' read', ' question', ' book', ' game'],
       [' player', ' writer', ' athlete', ' student', ' shooter',
        ' coach', ' hitter', ' defender', ' guy', ' broadcaster']],
      dtype='<U13')

# Relations

In [ ]:
occurs_data = pd.read_csv('occurs_after_1000.csv', usecols=["head_name", "rel", "tail_names"])
occurs_data.head(10)

,head_name,rel,tail_names
0,Post influenza vaccination encephalitis,occurs_after,Administration of influenza vaccine
1,Basal cell carcinoma recurrent following cryos...,occurs_after,Cryosurgery
2,Adverse effect from PUVA photochemotherapy,occurs_after,Light therapy || Photochemotherapy with psoral...
3,Allergy to pea,occurs_after,Allergic sensitization
4,Bite of unidentified snake with neurological s...,occurs_after,Animal bite
5,Allergy to hypothalamic hormone,occurs_after,Allergic sensitization
6,Late effect of accidental injury,occurs_after,Traumatic injury
7,Radiotherapy scar,occurs_after,Procedure || Radiation oncology AND/OR radioth...
8,Atonic postpartum hemorrhage,occurs_after,Delivery procedure
9,Late effect of skin and subcutaneous tissue in...,occurs_after,Injury || Traumatic injury


In [ ]:
# Prompts to use
rel_name = occurs_data['rel'][0]
default_prompt = prompts.loc[prompts['pid'] == rel_name]['default_prompt'].tolist()[0]
human_prompt  = prompts.loc[prompts['pid'] == rel_name]['human_prompt'].tolist()[0]
print(f"Default prompt: {default_prompt}\nHuman prompt: {human_prompt}")

Default prompt: [X] occurs after [Y] .
Human prompt: [X] occurs after [Y] .


In [ ]:
# Preparing inputs
def prepare_inputs(data, prompt:str):
  # number of rows
  n = len(data)

  inputs = [prompt for _ in range(n)]
  heads = data['head_name'].tolist()

  inputs = [input.replace('[X]', head) for input, head in zip(inputs, heads)]
  inputs = [input.replace('[Y] ', '[MASK]') for input in inputs]
  return inputs

In [ ]:
inputs = prepare_inputs(occurs_data, default_prompt)
print(inputs[:10])

['Post influenza vaccination encephalitis occurs after [MASK].', 'Basal cell carcinoma recurrent following cryosurgery occurs after [MASK].', 'Adverse effect from PUVA photochemotherapy occurs after [MASK].', 'Allergy to pea occurs after [MASK].', 'Bite of unidentified snake with neurological signs occurs after [MASK].', 'Allergy to hypothalamic hormone occurs after [MASK].', 'Late effect of accidental injury occurs after [MASK].', 'Radiotherapy scar occurs after [MASK].', 'Atonic postpartum hemorrhage occurs after [MASK].', 'Late effect of skin and subcutaneous tissue injury occurs after [MASK].']


In [ ]:
# Getting predictions
pred_1 = get_model_predictions(model_checkpoint=biomedbert_models['BioMedBERT_base_abstract'], inputs=inputs, top_n=1)
pred_1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


array([['vaccination'],
       ['years'],
       ['transplantation'],
       ['birth'],
       ['birth'],
       ['surgery'],
       ['birth'],
       ['radiotherapy'],
       ['delivery'],
       ['surgery'],
       ['trauma'],
       ['stroke'],
       ['surgery'],
       ['trauma'],
       ['injection'],
       ['surgery'],
       ['surgery'],
       ['transplantation'],
       ['surgery'],
       ['trauma'],
       ['surgery'],
       ['transplantation'],
       ['surgery'],
       ['immunization'],
       ['chemotherapy'],
       ['cholecystectomy'],
       ['pregnancy'],
       ['esophagectomy'],
       ['varicella'],
       ['treatment'],
       ['surgery'],
       ['surgery'],
       ['cesarean'],
       ['laminectomy'],
       ['surgery'],
       ['trauma'],
       ['colonoscopy'],
       ['catheterization'],
       ['surgery'],
       ['surgery'],
       ['surgery'],
       ['ingestion'],
       ['surgery'],
       ['injection'],
       ['delivery'],
       ['surgery'],
     

In [ ]:
tails = occurs_data['tail_names'].tolist()
tails = list(map(lambda x: x.split(' || '), tails))
tails

[['Administration of influenza vaccine'],
 ['Cryosurgery'],
 ['Light therapy', 'Photochemotherapy with psoralens and ultraviolet A'],
 ['Allergic sensitization'],
 ['Animal bite'],
 ['Allergic sensitization'],
 ['Traumatic injury'],
 ['Procedure', 'Radiation oncology AND/OR radiotherapy'],
 ['Delivery procedure'],
 ['Injury', 'Traumatic injury'],
 ['Fat necrosis'],
 ['Spontaneous cerebral hemorrhage'],
 ['Transplantation',
  'Implantation of prosthetic device',
  'Surgical construction of arteriovenous shunt'],
 ['Injury of knee', 'Traumatic event'],
 ['Procedure', 'Injection'],
 ['Colostomy', 'Procedure'],
 ['Procedure'],
 ['Corneal transplant'],
 ['Allergic sensitization'],
 ['Traumatic injury', 'Traumatic event'],
 ['Extraction of cataract', 'Implantation of phakic intraocular lens implant'],
 ['Transplantation of bone marrow', 'Grafting procedure'],
 ['Allergic sensitization'],
 ['Active or passive immunization'],
 ['Allergic sensitization'],
 ['Implantation of prosthetic device', 

In [ ]:
# Computing top_n accuracy - percentage of examples where one of the top n answers is in the set of correct tails (ignore case)
def compute_accuracy(predictions, tails, verbose=0):
  hits = 0
  n = len(predictions)

  for i in range(n):
    preds = [prediction.strip().lower() for prediction in predictions[i]]
    # strip - RoBERTa models predict extra space at the beginning
    tls = [tail.strip().lower() for tail in tails[i]]

    if verbose==1:
      print(f"Predictions: {preds}")
      print(f"True answers: {tls}")

    if set(preds).intersection(tls):
      hits += 1
      if verbose:
        print("\033[1mTrue prediction!\033[0m")

  return (hits/n)*100

In [ ]:
top_1_acc = compute_accuracy(pred_1, tails)
print(f'Top 1 accuracy: {top_1_acc:.2f} %')

Top 1 accuracy: 4.00 %


In [ ]:
pred_10 = get_model_predictions(model_checkpoint=biomedbert_models['BioMedBERT_base_full'], inputs=inputs, top_n=10)
pred_10

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

array([['vaccination', 'birth', 'immunization', 'childhood',
        'vaccinations', 'infection', 'hospitalization', 'delivery',
        'influenza', 'seroconversion'],
       ['radiotherapy', 'years', 'surgery', 'chemotherapy', 'decades',
        'months', 'recurrence', 'treatment', 'excision', 'irradiation'],
       ['surgery', 'radiotherapy', 'treatment', 'chemotherapy',
        'irradiation', 'menopause', 'delivery', 'birth', 'radiation',
        'therapy'],
       ['surgery', 'birth', 'pregnancy', 'exercise', 'childbirth',
        'childhood', 'weaning', 'vaccination', 'transplantation',
        'trauma'],
       ['childhood', 'trauma', 'birth', 'childbirth', 'accidents',
        'death', 'delivery', 'adolescence', 'surgery', 'ingestion'],
       ['birth', 'childbirth', 'surgery', 'puberty', 'trauma',
        'childhood', 'menopause', 'vaccination', 'pregnancy',
        'splenectomy'],
       ['surgery', 'treatment', 'discharge', 'trauma', 'operation',
        'recovery', 'hospita

In [ ]:
top_10_acc = compute_accuracy(pred_10, tails)
print(f'Top 10 accuracy: {top_10_acc:.2f} %')

Top 10 accuracy: 12.10 %


In [ ]:
def cosine_similarity(v1, v2):
    v1 = np.array(v1)
    v2 = np.array(v2)

    dot_product = np.dot(v1, v2)

    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)

    cosine_sim = dot_product / (norm_v1 * norm_v2)

    return cosine_sim

print(f'Cosine Similarity: {cosine_similarity([1,2,3], [4,5,6])}')

Cosine Similarity: 0.9746318461970762


In [ ]:
model_name = bert_models['BERT_base']
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name, from_pt=True, output_hidden_states=True)

input_texts = [
    "The cat sat on the mat.",
    "The dog sat on the mat.",
    "The kitten sat on the mat.",
    "The mailman sat on the mat."
    ]

inputs = tokenizer(input_texts, return_tensors="tf", padding=True, truncation=True)

print(f"Tokens: {inputs['input_ids']}")
for i, input_ids in enumerate(inputs['input_ids']):
    print(f"Decoded tokens for sentence {i}: {tokenizer.decode(input_ids)}")

outputs = model(**inputs)

hidden_states = outputs.hidden_states

if not isinstance(hidden_states, tuple):
    raise ValueError("Model configuration does not support returning hidden states!")

# combining last 4 (TRY OTHER) layers for more robust results
last_four_layers = [hidden_states[i] for i in range(-4, 0)]
combined_layers = tf.reduce_mean(tf.stack(last_four_layers), axis=0)

# normalizing the combined embeddings
normalized_embeddings = tf.nn.l2_normalize(combined_layers, axis=2)

# position of mask token in original sentence
third_token_embeddings = normalized_embeddings[:, 2, :]

print(third_token_embeddings)

# compare word embeddings ()
print(f"Cosine similarity (\"cat\", \"dog\"): {cosine_similarity(tf.reshape(third_token_embeddings[0], shape=(1, -1)), tf.reshape(third_token_embeddings[1], shape=(-1, 1)))[0][0]}")
print(f"Cosine similarity (\"cat\", \"kitten\"): {cosine_similarity(tf.reshape(third_token_embeddings[0], shape=(1, -1)), tf.reshape(third_token_embeddings[2], shape=(-1, 1)))[0][0]}")
print(f"Cosine similarity (\"dog\", \"kitten\"): {cosine_similarity(tf.reshape(third_token_embeddings[1], shape=(1, -1)), tf.reshape(third_token_embeddings[2], shape=(-1, 1)))[0][0]}")
print(f"Cosine similarity (\"cat\", \"mailman\"): {cosine_similarity(tf.reshape(third_token_embeddings[0], shape=(1, -1)), tf.reshape(third_token_embeddings[3], shape=(-1, 1)))[0][0]}")
print(f"Cosine similarity (\"dog\", \"mailman\"): {cosine_similarity(tf.reshape(third_token_embeddings[1], shape=(1, -1)), tf.reshape(third_token_embeddings[3], shape=(-1, 1)))[0][0]}")
print(f"Cosine similarity (\"kitten\", \"mailman\"): {cosine_similarity(tf.reshape(third_token_embeddings[2], shape=(1, -1)), tf.reshape(third_token_embeddings[3], shape=(-1, 1)))[0][0]}")

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Tokens: [[  101  1996  4937  2938  2006  1996 13523  1012   102     0]
 [  101  1996  3899  2938  2006  1996 13523  1012   102     0]
 [  101  1996 18401  2938  2006  1996 13523  1012   102     0]
 [  101  1996  5653  2386  2938  2006  1996 13523  1012   102]]
Decoded tokens for sentence 0: [CLS] the cat sat on the mat. [SEP] [PAD]
Decoded tokens for sentence 1: [CLS] the dog sat on the mat. [SEP] [PAD]
Decoded tokens for sentence 2: [CLS] the kitten sat on the mat. [SEP] [PAD]
Decoded tokens for sentence 3: [CLS] the mailman sat on the mat. [SEP]
tf.Tensor(
[[-0.01194338 -0.0051559   0.0104109  ... -0.02537592  0.03168032
   0.04000831]
 [ 0.02839966  0.02874123  0.00292385 ... -0.04067679  0.02913046
   0.03197852]
 [-0.04018649 -0.03780384 -0.012027   ... -0.03123694  0.027043
  -0.03035366]
 [ 0.05976407 -0.01487961  0.02051217 ...  0.01845756  0.0075776
  -0.05614597]], shape=(4, 768), dtype=float32)
Cosine similarity ("cat", "dog"): 0.8437002897262573
Cosine similarity ("cat", "k

In [ ]:
def replace_masks(model_checkpoint, masked_inputs, tails):

  n = len(masked_inputs)
  full_sentences = []

  mask_string = '[MASK]'

  if 'roberta' in model_checkpoint:
    mask_string = '<mask>'

  for i in range(n):
    # strip - RoBERTa models predict extra space at the beginning
   example_sentences = [masked_inputs[i].replace(mask_string, tail.strip().lower()) for tail in tails[i]]
   full_sentences.append(example_sentences)

  return full_sentences

print(replace_masks(biomedbert_models['BioMedBERT_base_full'], inputs[:10], tails[:10]))
print(replace_masks(biomedbert_models['BioMedBERT_base_full'], inputs[:10], pred_10[:10]))

[['Post influenza vaccination encephalitis occurs after administration of influenza vaccine.'], ['Basal cell carcinoma recurrent following cryosurgery occurs after cryosurgery.'], ['Adverse effect from PUVA photochemotherapy occurs after light therapy.', 'Adverse effect from PUVA photochemotherapy occurs after photochemotherapy with psoralens and ultraviolet a.'], ['Allergy to pea occurs after allergic sensitization.'], ['Bite of unidentified snake with neurological signs occurs after animal bite.'], ['Allergy to hypothalamic hormone occurs after allergic sensitization.'], ['Late effect of accidental injury occurs after traumatic injury.'], ['Radiotherapy scar occurs after procedure.', 'Radiotherapy scar occurs after radiation oncology and/or radiotherapy.'], ['Atonic postpartum hemorrhage occurs after delivery procedure.'], ['Late effect of skin and subcutaneous tissue injury occurs after injury.', 'Late effect of skin and subcutaneous tissue injury occurs after traumatic injury.']]
[

In [ ]:
# 1. obtain embeddings of the expected token and the predicted token from the word embedding layer of a model
# 2. compute cosine similarity metric (TO DO: Multi Token scenario: Averaging cosine similarities, BLEU score (???))

# single token scenario: a) only first token of true answer taken into consideration
def compute_cos_sim_accuracy_v1(model_checkpoint:str, original_inputs, predictions, tails, verbose=0):
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
  model = TFAutoModel.from_pretrained(model_checkpoint, from_pt=True, output_hidden_states=True)

  # Adjusting inputs for RoBERTa models
  if 'roberta' in model_checkpoint:
    original_inputs = [change_input_format(input) for input in original_inputs]

  # model_max_length field not set by default for BioBERT and BioMedBERT models
  if 'bio' in model_checkpoint.lower():
    tokenizer.model_max_length = 512

  # Tokenizing original inputs
  tokenized_inputs = tokenizer(original_inputs, return_tensors="tf", padding=True, truncation=True) # max_length=128 (by default max_length = tokenizer.model_max_length)

  # Getting mask_token indexes for every example
  mask_token_indexes = []

  for i, input in enumerate(tokenized_inputs["input_ids"]):
                                                                                   # not necessary (all tokenizers have mask_token_id defined)
    mask_token_id = tokenizer.mask_token_id if tokenizer.mask_token_id is not None else tokenizer.convert_tokens_to_ids(["[MASK]"])[0]

    mask_token_indexes.append(np.argwhere(tokenized_inputs["input_ids"].numpy()[i] == mask_token_id)[0, 0])


  predicted_sentences = replace_masks(model_checkpoint, original_inputs, predictions)
  true_sentences = replace_masks(model_checkpoint, original_inputs, tails)

  hits = 0
  n = len(original_inputs)
  # for every example in relation dataset
  for i in range(n):

    predicted_sentences_tkz = tokenizer(predicted_sentences[i], return_tensors="tf", padding=True, truncation=True)

    hidden_states = model(**predicted_sentences_tkz).hidden_states

    if not isinstance(hidden_states, tuple):
      raise ValueError("Model configuration does not support returning hidden states!")

    # combining last 4 (TRY OTHER) layers for more robust results - experiment with combining embeddings from earlier layers (???), Dimensionality reduction - PCA (???)
    combined_last_four_layers = tf.reduce_mean(tf.stack([hidden_states[i] for i in range(-4, 0)]), axis=0)
    # normalizing the combined embeddings
    pred_embeddings = tf.nn.l2_normalize(combined_last_four_layers, axis=2) [:, mask_token_indexes[i], :]

    true_sentences_tkz = tokenizer(true_sentences[i], return_tensors="tf", padding=True, truncation=True)
    hidden_states = model(**true_sentences_tkz).hidden_states
    # combining last 4 (TRY OTHER) layers for more robust results
    combined_last_four_layers = tf.reduce_mean(tf.stack([hidden_states[i] for i in range(-4, 0)]), axis=0)
    # normalizing the combined embeddings
    true_embeddings = tf.nn.l2_normalize(combined_last_four_layers, axis=2) [:, mask_token_indexes[i], :]

    highest_similarity = -1

    if verbose:
      most_similar_pred = None
      most_similar_true = None

    for j in range(len(pred_embeddings)):
      for k in range(len(true_embeddings)):

        if verbose:
          pred_token = tokenizer.decode(predicted_sentences_tkz["input_ids"][j].numpy()[mask_token_indexes[i]])
          true_token = tokenizer.decode(true_sentences_tkz["input_ids"][k].numpy()[mask_token_indexes[i]])

        similarity = cosine_similarity(tf.reshape(pred_embeddings[j], shape=(1, -1)), tf.reshape(true_embeddings[k], shape=(-1, 1)))[0][0]

        if verbose:
          # some words from true answers cut into multiple tokens (!!!)
          print(f"Cosine similarity between '{pred_token}' and '{true_token}': {similarity}")

        if similarity > highest_similarity:
            highest_similarity = similarity
            if verbose:
              most_similar_pred = pred_token
              most_similar_true = true_token

    if verbose:
      print(f"\033[1mHighest similarity is between '{most_similar_pred}' and '{most_similar_true}': {highest_similarity}\033[0m")
      print('--------------------------------------------')

    # 0.95 - threshold (can be parameter) - TRY OTHER
    if highest_similarity >= 0.95:
      hits += 1

  return (hits/n) * 100

# print(compute_cos_sim_accuracy_v1(biomedbert_models['BioMedBERT_base_full'], inputs[:10], pred_10[:10], tails[:10], verbose=1))
pred_5_roberta = get_model_predictions(model_checkpoint=roberta_models['RoBERTa_base'], inputs=inputs[:10], top_n=5)
print(compute_cos_sim_accuracy_v1(roberta_models['RoBERTa_base'], inputs[:10], pred_5_roberta[:10], tails[:10], verbose=1))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Cosine similarity between ' vaccination' and ' administration': 0.8891960978507996
Cosine similarity between ' vaccinations' and ' administration': 0.8829200863838196
Cosine similarity between ' infection' and ' administration': 0.8946338891983032
Cosine similarity between ' exposure' and ' administration': 0.9050487875938416
Cosine similarity between ' influenza' and ' administration': 0.8568709492683411
Highest similarity is between ' exposure' and ' administration': 0.9050487875938416
--------------------------------------------
Cosine similarity between ' surgery' and ' cry': 0.8358896374702454
Cosine similarity between ' chemotherapy' and ' cry': 0.858271598815918
Cosine similarity between ' treatment' and ' cry': 0.8431753516197205
Cosine similarity between ' death' and ' cry': 0.8333811163902283
Cosine similarity between ' diagnosis' and ' cry': 0.8337224125862122
Highest similarity is between ' chemotherapy' and ' cry': 0.858271598815918
----------------------------------------

In [ ]:
top_1_cos_acc = compute_cos_sim_accuracy_v1(biomedbert_models['BioMedBERT_base_abstract'], inputs, pred_1, tails)
print(f'Top 1 accuracy: {top_1_cos_acc:.2f} %')

Top 1 accuracy: 16.60 %


In [ ]:
top_10_cos_acc = compute_cos_sim_accuracy_v1(biomedbert_models['BioMedBERT_base_full'], inputs, pred_10, tails)
print(f'Top 10 accuracy: {top_10_cos_acc:.2f} %')

Top 10 accuracy: 37.30 %


In [ ]:
# single token scenario: b) combine embeddings of all of tokens of the answer - from mask_index to end ('.' token) - Problem: different granularity between predictions and true phrases
def compute_cos_sim_accuracy_v2(model_checkpoint:str, original_inputs, predictions, tails, verbose=0):
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
  model = TFAutoModel.from_pretrained(model_checkpoint, from_pt=True, output_hidden_states=True)

  # Adjusting inputs for RoBERTa models
  if 'roberta' in model_checkpoint:
    original_inputs = [change_input_format(input) for input in original_inputs]

  # model_max_length field not set by default for BioBERT and BioMedBERT models
  if 'bio' in model_checkpoint.lower():
    tokenizer.model_max_length = 512

  # Tokenizing original inputs
  tokenized_inputs = tokenizer(original_inputs, return_tensors="tf", padding=True, truncation=True) # max_length=128 (by default max_length = tokenizer.model_max_length)
  #print(tokenized_inputs['input_ids'])

  # Getting mask_token indexes for every example
  mask_token_indexes = []

  for i, input in enumerate(tokenized_inputs["input_ids"]):
                                                                                   # not necessary (all tokenizers have mask_token_id defined)
    mask_token_id = tokenizer.mask_token_id if tokenizer.mask_token_id is not None else tokenizer.convert_tokens_to_ids(["[MASK]"])[0]

    mask_token_indexes.append(np.argwhere(tokenized_inputs["input_ids"].numpy()[i] == mask_token_id)[0, 0])

  predicted_sentences = replace_masks(model_checkpoint, original_inputs, predictions)
  true_sentences = replace_masks(model_checkpoint, original_inputs, tails)

  hits = 0
  n = len(original_inputs)
  # for every example in relation dataset
  for i in range(n):

    predicted_sentences_tkz = tokenizer(predicted_sentences[i], return_tensors="tf", padding=True, truncation=True)

    hidden_states = model(**predicted_sentences_tkz).hidden_states

    if not isinstance(hidden_states, tuple):
      raise ValueError("Model configuration does not support returning hidden states!")

    # combining last 4 (TRY OTHER) layers for more robust results - experiment with combining embeddings from earlier layers (???), Dimensionality reduction - PCA (???)
    combined_last_four_layers = tf.reduce_mean(tf.stack([hidden_states[i] for i in range(-4, 0)]), axis=0)

    # normalizing the combined embeddings
    pred_embeddings = tf.nn.l2_normalize(combined_last_four_layers, axis=2) [:, mask_token_indexes[i], :]

    true_sentences_tkz = tokenizer(true_sentences[i], return_tensors="tf", padding=True, truncation=True)
    hidden_states = model(**true_sentences_tkz).hidden_states

    # combining last 4 layers for more robust results
    combined_last_four_layers = tf.reduce_mean(tf.stack([hidden_states[i] for i in range(-4, 0)]), axis=0)

    # finding end indexes of the true phrases (position of the period token '.' - different for every true phrase!)
    period_token_id = tokenizer.convert_tokens_to_ids(".")
    end_indexes = []
    embedding_size = combined_last_four_layers.shape[2]
    true_embeddings = tf.zeros((0, embedding_size))

    for m in range(len(true_sentences_tkz["input_ids"])):

      end_index = np.where(true_sentences_tkz["input_ids"].numpy()[m] == period_token_id)[0]

      if len(end_index) > 0:
        end_index = end_index[0]
      else:
       # in case there's no '.', take until the some special token occurs
        end_index = np.where(true_sentences_tkz["input_ids"].numpy()[m] in tokenizer.all_special_ids)[0][0]

      # extracting the embeddings from the mask_token_indexes[i] (start index) to the end index
      phrase_embedding = combined_last_four_layers[m, mask_token_indexes[i]:end_index, :]

      # normalizing and combining the embeddings of the whole phrase
      true_embedding = tf.nn.l2_normalize(tf.reduce_mean(phrase_embedding, axis=0), axis=0)

      true_embeddings = tf.concat([true_embeddings, tf.reshape(true_embedding, (1, -1))], axis=0)

      end_indexes.append(end_index)


    highest_similarity = -1

    if verbose:
      most_similar_pred = None
      most_similar_true = None

    for j in range(len(pred_embeddings)):
      for k in range(len(true_embeddings)):

        if verbose:
          pred_token = tokenizer.decode(predicted_sentences_tkz["input_ids"][j].numpy()[mask_token_indexes[i]])
          true_phrase = tokenizer.decode(true_sentences_tkz["input_ids"][k].numpy()[mask_token_indexes[i]:end_indexes[k]])

        similarity = cosine_similarity(tf.reshape(pred_embeddings[j], shape=(1, -1)), tf.reshape(true_embeddings[k], shape=(-1, 1)))[0][0]

        if verbose:
          # some words from true answers cut into multiple tokens (!!!)
          print(f"Cosine similarity between '{pred_token}' and '{true_phrase}': {similarity}")

        if similarity > highest_similarity:
            highest_similarity = similarity
            if verbose:
              most_similar_pred = pred_token
              most_similar_true = true_phrase

    if verbose:
      print(f"\033[1mHighest similarity is between '{most_similar_pred}' and '{most_similar_true}': {highest_similarity}\033[0m")
      print('--------------------------------------------')

    # 0.95 - threshold (can be function parameter) - TRY OTHER
    if highest_similarity >= 0.95:
      hits += 1

  return (hits/n) * 100

pred_5_roberta = get_model_predictions(model_checkpoint=roberta_models['RoBERTa_large'], inputs=inputs[:10], top_n=5)
print(compute_cos_sim_accuracy_v2(roberta_models['RoBERTa_large'], inputs[:10], pred_5_roberta[:10], tails[:10], verbose=1))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Cosine similarity between ' vaccination' and ' administration of influenza vaccine': 0.9667031764984131
Cosine similarity between ' infection' and ' administration of influenza vaccine': 0.933690071105957
Cosine similarity between ' treatment' and ' administration of influenza vaccine': 0.9394055008888245
Cosine similarity between ' exposure' and ' administration of influenza vaccine': 0.9482665061950684
Cosine similarity between ' vaccinations' and ' administration of influenza vaccine': 0.954810619354248
Highest similarity is between ' vaccination' and ' administration of influenza vaccine': 0.9667031764984131
--------------------------------------------
Cosine similarity between ' chemotherapy' and ' cryosurgery': 0.8986359238624573
Cosine similarity between ' treatment' and ' cryosurgery': 0.9033290147781372
Cosine similarity between ' surgery' and ' cryosurgery': 0.9313793778419495
Cosine similarity between ' years' and ' cryosurgery': 0.8706271648406982
Cosine similarity between 

In [ ]:
pred_5_biomed = get_model_predictions(model_checkpoint=biomedbert_models['BioMedBERT_base_full'], inputs=inputs[:20], top_n=5)
print(compute_cos_sim_accuracy_v2(biomedbert_models['BioMedBERT_base_full'], inputs[:20], pred_5_biomed[:20], tails[:20], verbose=1))

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Cosine similarity between 'vaccination' and 'administration of influenza vaccine': 0.9442157745361328
Cosine similarity between 'birth' and 'administration of influenza vaccine': 0.8761966824531555
Cosine similarity between 'immunization' and 'administration of influenza vaccine': 0.9306254386901855
Cosine similarity between 'childhood' and 'administration of influenza vaccine': 0.8729021549224854
Cosine similarity between 'vaccinations' and 'administration of influenza vaccine': 0.9219234585762024
Highest similarity is between 'vaccination' and 'administration of influenza vaccine': 0.9442157745361328
--------------------------------------------
Cosine similarity between 'radiotherapy' and 'cryosurgery': 0.9210992455482483
Cosine similarity between 'years' and 'cryosurgery': 0.8444016575813293
Cosine similarity between 'surgery' and 'cryosurgery': 0.9139726161956787
Cosine similarity between 'chemotherapy' and 'cryosurgery': 0.897855818271637
Cosine similarity between 'decades' and 'c

In [ ]:
pred_1_biomed = get_model_predictions(model_checkpoint=biomedbert_models['BioMedBERT_base_full'], inputs=inputs[:200], top_n=1)
top_1_cos_acc = compute_cos_sim_accuracy_v2(biomedbert_models['BioMedBERT_base_full'], inputs[:200], pred_1_biomed[:200], tails[:200])
print(f'Top 1 accuracy: {top_1_cos_acc:.2f} %')

Top 1 accuracy: 17.00 %


# MLM acccuracy measuring function

In [ ]:
                                                    # TO DO - Pass as Callable
def compute_mml_top_n_accuracy(model_checkpoint:str, accuracy_function:str, relation_dataset:str, dataset_frac=1 , top_n=5, random_state=123, verbose=0):

  data = pd.read_csv(relation_dataset, usecols=["head_name", "rel", "tail_names"])
  # For quicker testing due to resource limitations
  data_chunk = data.sample(frac=dataset_frac, random_state=random_state).reset_index(drop=True)

  rel_name = data_chunk['rel'][0]
  default_prompt = prompts.loc[prompts['pid'] == rel_name]['default_prompt'].tolist()[0]

  model_inputs = prepare_inputs(data_chunk, default_prompt)

  predicted_objects = get_model_predictions(model_checkpoint=model_checkpoint, inputs=model_inputs, top_n=top_n, verbose=0)

  true_objects = data_chunk['tail_names'].tolist()
  true_objects = list(map(lambda x: x.split(' || '), true_objects))

  if accuracy_function not in ['exact', 'cosine']:
    raise ValueError("Accuracy must be either 'exact' or 'cosine'.")

  if accuracy_function == 'exact':
    return compute_accuracy(predicted_objects, true_objects, verbose)
  elif accuracy_function == 'cosine':
    return compute_cos_sim_accuracy_v1(model_checkpoint, model_inputs, predicted_objects, true_objects, verbose)
  elif accuracy_function == 'cosine2':
    return compute_cos_sim_accuracy_v2(model_checkpoint, model_inputs, predicted_objects, true_objects, verbose)

In [ ]:
# Comparing BERT models top 5 exact accuracy
print('Top 5 accuracy')

relation = 'occurs_after'
print(f"Relation: \033[1m{relation}\033[0m")

for k, v in bert_models.items():
  acc = compute_mml_top_n_accuracy(v, 'exact', f'{relation}_1000.csv', top_n=5)
  print(f"{k}: {acc:.2f}%\n")

Top 5 accuracy
Relation: occurs_after


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

BERT_large_wwm: 6.40%



In [ ]:
for k, v in roberta_models.items():
  acc = compute_mml_top_n_accuracy(v, 'exact', f'{relation}_1000.csv', top_n=5)
  print(f"{k}: {acc:.2f}%\n")

RoBERTa_base: 2.60%

RoBERTa_large: 4.10%



In [ ]:
for k, v in albert_models.items():
  acc = compute_mml_top_n_accuracy(v, 'exact', f'{relation}_1000.csv', top_n=5)
  print(f"{k}: {acc:.2f}%\n")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/710 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/893M [00:00<?, ?B/s]

ALBERT_xxlarge: 8.60%



In [ ]:
for k, v in biobert_models.items():
  acc = compute_mml_top_n_accuracy(v, 'exact', f'{relation}_1000.csv', top_n=5)
  print(f"{k}: {acc:.2f}%\n")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

BioBERT: 3.10%



In [ ]:
for k, v in biomedbert_models.items():
  acc = compute_mml_top_n_accuracy(v, 'exact', f'{relation}_1000.csv', top_n=5)
  print(f"{k}: {acc:.2f}%\n")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/225k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/225k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

BioMedBERT_large_abstract: 11.20%



In [ ]:
# Comparing BERT models top 5 cos sim accuracy
print('Top 5 accuracy')

relation = 'occurs_after'
print(f"Relation: \033[1m{relation}\033[0m")

for k, v in bert_models.items():
  acc = compute_mml_top_n_accuracy(v, 'cosine', f'{relation}_1000.csv', top_n=5)
  print(f"{k}: {acc:.2f}%\n")

Top 5 accuracy
Relation: occurs_after
BERT_base: 6.10

BERT_large: 9.80

BERT_large_wwm: 8.00



In [ ]:
for k, v in roberta_models.items():
  acc = compute_mml_top_n_accuracy(v, 'cosine', f'{relation}_1000.csv', top_n=5)
  print(f"{k}: {acc:.2f}%\n")

RoBERTa_base: 27.00



tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

RoBERTa_large: 24.00



In [ ]:
for k, v in albert_models.items():
  acc = compute_mml_top_n_accuracy(v, 'cosine', f'{relation}_1000.csv', top_n=5)
  print(f"{k}: {acc:.2f}%\n")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/710 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/893M [00:00<?, ?B/s]

ALBERT_xxlarge: 9.00



In [ ]:
for k, v in biobert_models.items():
  acc = compute_mml_top_n_accuracy(v, 'cosine', f'{relation}_1000.csv', top_n=5)
  print(f"{k}: {acc:.2f}%\n")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

BioBERT: 7.00



In [ ]:
for k, v in biomedbert_models.items():
  acc = compute_mml_top_n_accuracy(v, 'cosine', f'{relation}_1000.csv', top_n=5)
  print(f"{k}: {acc:.2f}%\n")

BioMedBERT_base_abstract: 11.00

BioMedBERT_base_full: 35.00



tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/225k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

BioMedBERT_large_abstract: 36.00



# Multi-token issue

In [64]:
# Conditional MLM
# filling masks in parallel independently (Independent approach)
# TO DO - top_n predictions (how to combine predictions of particular tokens (???))
def fill_masks_independently(model_checkpoint: str, inputs: list[str], top_n=5, verbose=0):
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint, from_pt=True)

    # Adjusting inputs for RoBERTa models
    if 'roberta' in model_checkpoint:
        inputs = [change_input_format(input) for input in inputs]

    # model_max_length field not set by default for BioBERT and BioMedBERT models
    if 'bio' in model_checkpoint.lower():
        tokenizer.model_max_length = 512

    if verbose:
        print(f'Chosen model: {model_checkpoint}')
        model.summary()

    tokenized_inputs = tokenizer(inputs, return_tensors="tf", padding=True, truncation=True)

    # getting the token logits from the model
    token_logits = model(**tokenized_inputs).logits

    outputs = []
    outputs_decoded = []

    for i, input in enumerate(inputs):


        input_ids = tokenized_inputs["input_ids"].numpy()[i]

        # finding all positions of the [MASK] token
        mask_token_id = tokenizer.mask_token_id
        mask_token_indices = np.where(input_ids == mask_token_id)[0]
        #print(mask_token_indices)

        prediction = []
        for mask_token_index in mask_token_indices:

          mask_token_logits = token_logits[i, mask_token_index, :]

          top_token = np.argmin(-mask_token_logits.numpy())

          prediction.append(top_token)

        outputs.append(prediction)
        prediction_decoded = tokenizer.decode(prediction, skip_special_tokens=True)
        outputs_decoded.append(prediction_decoded)

    return outputs, outputs_decoded

In [65]:
# tokenizer.convert_ids_to_tokens:
#This method converts each token ID into its corresponding token string.
#It returns a list of token strings.
#It retains the special tokens and subword tokens as they are

#tokenizer.decode:
#This method converts a list of token IDs into a single string.
#It concatenates the token strings into a coherent sentence.
#It handles subwords properly by removing special characters (like ## in BERT) and joining subwords.
#It removes special tokens by default

test_input = ["Paris is [MASK] [MASK] to visit.", "Jupiter is the largest planet of the [MASK] [MASK].", "The weather forecast predicts [MASK] [MASK] for tomorrow.", "The weather forecast predicts heavy rain and [MASK] [MASK].", "He wanted to visit the museum and explore the [MASK] [MASK].", "She was excited about the promotion and [MASK] [MASK].", "He is known for his dedication and [MASK] [MASK] [MASK].", "They plan to travel to Italy and enjoy the beautiful [MASK] [MASK] [MASK].",  "She decided to go to the market and buy some fresh [MASK] [MASK] [MASK] [MASK].", "He set a new world record in the [MASK] [MASK] [MASK] [MASK] event."]
outputs, outputs_dec = fill_masks_independently(bert_models['BERT_base'], test_input)

test_tokenizer = AutoTokenizer.from_pretrained(bert_models['BERT_base'])
# getting less coherent and contextually appropriate predictions as the sequence of [MASK] tokens becomes longer
for output, output_dec in zip(outputs, outputs_dec):
  print(f"{output} : {output_dec} : {test_tokenizer.convert_ids_to_tokens(output)}")

[2036, 4569] : also fun : ['also', 'fun']
[3103, 2291] : sun system : ['sun', 'system']
[1996, 4633] : the weather : ['the', 'weather']
[4586, 7266] : snow winds : ['snow', 'winds']
[2334, 2088] : local world : ['local', 'world']
[1996, 2769] : the money : ['the', 'money']
[2010, 3167, 3241] : his personal thinking : ['his', 'personal', 'thinking']
[3059, 3059, 10833] : italian italian countryside : ['italian', 'italian', 'countryside']
[11546, 1998, 2000, 2014] : vegetables and to her : ['vegetables', 'and', 'to', 'her']
[2230, 3816, 3783, 8846] : 2010 mixed olympics relay : ['2010', 'mixed', 'olympics', 'relay']


In [74]:
# Conditional MLM
# filling masks in parallel autoregressively (Order approach - left to right)
# TO DO - top_n predictions
from transformers import AutoTokenizer, TFAutoModelForMaskedLM
import tensorflow as tf

def fill_masks_autoregressively(model_checkpoint: str, inputs: list[str], top_n=5, verbose=0):
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint, from_pt=True)

    # Adjusting inputs for RoBERTa models
    if 'roberta' in model_checkpoint:
        inputs = [change_input_format(input) for input in inputs]

    # model_max_length field not set by default for BioBERT and BioMedBERT models
    if 'bio' in model_checkpoint.lower():
        tokenizer.model_max_length = 512

    if verbose:
        print(f'Chosen model: {model_checkpoint}')
        model.summary()

    outputs = []
    outputs_decoded = []

    for input_text in inputs:
        print(input_text)
        tokenized_input = tokenizer(input_text, return_tensors="tf")
        #print(tokenized_input)

        # Initialize the input_ids with the token IDs of the input
        input_ids = tokenized_input["input_ids"]
        input_ids_np = input_ids.numpy()[0]
        print(f"input_ids: {input_ids_np}")


        # Find all positions of the [MASK] tokens
        mask_token_id = tokenizer.mask_token_id
        mask_token_indices = np.where(input_ids_np == mask_token_id)[0]
        #print(f"mask position: {mask_token_indices}")

        prediction = []
        for mask_index in mask_token_indices:


          token_logits = model(**tokenized_input).logits[0]
          #print(token_logits.shape)
          mask_token_logits = token_logits[mask_index, :]
          #print(mask_token_logits.shape)

          # getting the top predicted token
          top_token = np.argmin(-mask_token_logits.numpy())
          # print(f" {top_token}: {tokenizer.convert_ids_to_tokens([top_token])} ")

          prediction.append(top_token)
          input_ids_np[mask_index] = top_token
          print(f"input_ids: {input_ids_np}")

          tokenized_input = tf.tensor_scatter_nd_update(input_ids, [mask_index], [top_token])

        # Decode the final output sequence
      #  output_ids = tf.squeeze(input_ids).numpy().tolist()
      #  output_text = tokenizer.decode(output_ids, skip_special_tokens=True)

     #   outputs.append(output_ids)
     #   outputs_decoded.append(output_text)

   # return outputs, outputs_decoded

In [75]:
fill_masks_autoregressively(bert_models['BERT_base'], test_input)

Paris is [MASK] [MASK] to visit.
input_ids: [ 101 3000 2003  103  103 2000 3942 1012  102]
input_ids: [ 101 3000 2003 2036  103 2000 3942 1012  102]


InvalidArgumentError: {{function_node __wrapped__TensorScatterUpdate_device_/job:localhost/replica:0/task:0/device:CPU:0}} The inner 1 dimensions of output.shape=[1,9] must match the inner 1 dimensions of updates.shape=[1] [Op:TensorScatterUpdate]

In [31]:
test_input = ["Paris is [MASK] [MASK] to visit.", "Jupiter is the largest planet of the [MASK] [MASK].", "The weather forecast predicts [MASK] [MASK] for tomorrow.", "The weather forecast predicts heavy rain and [MASK] [MASK].", "He wanted to visit the museum and explore the [MASK] [MASK].", "She was excited about the promotion and [MASK] [MASK].", "He is known for his dedication and [MASK] [MASK] [MASK].", "They plan to travel to Italy and enjoy the beautiful [MASK] [MASK] [MASK].",  "She decided to go to the market and buy some fresh [MASK] [MASK] [MASK] [MASK].", "He set a new world record in the [MASK] [MASK] [MASK] [MASK] event."]
outputs, outputs_dec = fill_masks_autoregressively(bert_models['BERT_base'], test_input)

test_tokenizer = AutoTokenizer.from_pretrained(bert_models['BERT_base'])

for output, output_dec in zip(outputs, outputs_dec):
  print(f"{output} : {output_dec} : {test_tokenizer.convert_ids_to_tokens(output)}")

[101, 3000, 2003, 1012, 1012, 2000, 3942, 1012, 102] : paris is.. to visit. : ['[CLS]', 'paris', 'is', '.', '.', 'to', 'visit', '.', '[SEP]']
[101, 13035, 2003, 1996, 2922, 4774, 1997, 1996, 1012, 1012, 1012, 102] : jupiter is the largest planet of the... : ['[CLS]', 'jupiter', 'is', 'the', 'largest', 'planet', 'of', 'the', '.', '.', '.', '[SEP]']
[101, 1996, 4633, 19939, 16014, 2015, 1012, 1012, 2005, 4826, 1012, 102] : the weather forecast predicts.. for tomorrow. : ['[CLS]', 'the', 'weather', 'forecast', 'predict', '##s', '.', '.', 'for', 'tomorrow', '.', '[SEP]']
[101, 1996, 4633, 19939, 16014, 2015, 3082, 4542, 1998, 1012, 1012, 1012, 102] : the weather forecast predicts heavy rain and... : ['[CLS]', 'the', 'weather', 'forecast', 'predict', '##s', 'heavy', 'rain', 'and', '.', '.', '.', '[SEP]']
[101, 2002, 2359, 2000, 3942, 1996, 2688, 1998, 8849, 1996, 1012, 1012, 1012, 102] : he wanted to visit the museum and explore the... : ['[CLS]', 'he', 'wanted', 'to', 'visit', 'the', 'muse

In [ ]:
# TO DO
def fill_masks_by_confidence(model_checkpoint: str, input_query: str, mask_token="[MASK]", top_n=5):

    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    tokenized_input = tokenizer(input_query, return_tensors="tf", padding=True, truncation=True)

    mask_token_id = tokenizer.mask_token_id if tokenizer.mask_token_id is not None else tokenizer.convert_tokens_to_ids([mask_token])[0]
    mask_indices = tf.where(tf.equal(tokenized_input["input_ids"], mask_token_id))

    model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)
    token_logits = model(**tokenized_input).logits

    predictions = []
    for mask_index in mask_indices:
        mask_position = mask_index[1].numpy()
        mask_logits = token_logits[0, mask_position, :]
        top_token_ids = tf.argsort(-mask_logits)[:top_n].numpy()
        top_tokens = tokenizer.decode(top_token_ids)
        predictions.append(top_tokens)

    return predictions

In [ ]:
fill_masks_independently(bert_models['BERT_base'], "Paris is [MASK][MASK] to visit.")

['also not always a definitely', 'fun easy pleasant welcome hard']

In [ ]:
fill_masks_autoregressively(bert_models['BERT_base'], "Paris is [MASK][MASK] to visit.")

In [ ]:
fill_masks_by_confidence(bert_models['BERT_base'], "Paris is [MASK][MASK] to visit.")

# Restricted candidate set